In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
import datetime as DT

In [ ]:
india_data=pd.read_csv('csv_files/India_yf.csv')
india_data_sym_list=list(india_data['symbol'])
india_data_isin_list=list(india_data['isin'])
india_data


In [ ]:
# mention dates and time_period

starting_date='2021-12-31'
end_date='2022-05-10'
T=7

# create master_df for all isin acroos all countries and master

winner_loser_df=[['company_name','isin','last_price','%_change']]
data_not_found=[]


# function for calculating perc_returns which takes yf df with all dates

def perc_return(df):
    close_today=df[df.index==starting_date].Close[0]
    T_day_before=df.tail(T).iloc[0]['Close']

    if(close_today==T_day_before):
        perc_change=0

    else:
        perc_change=(close_today-T_day_before)/(T_day_before)
        
    perc_change= np.ceil(perc_change*100)
    return perc_change

def create_win_los_df(df):
    df=pd.DataFrame(df)
    df.columns=df.iloc[0]
    df=df.drop(df.index[0])
    return df

In [ ]:
i=0
# get data from new year for one isin

for sym in india_data_sym_list[:10]:
    
    try:
        # get yahoo finance data
        stock_data=yf.download(sym, start=starting_date,end=end_date, progress=False,)
        print('getting data from yfinance')

        #    put all the dates in between and forward fill
        stock_data=stock_data.resample('1D').mean()
        stock_data['Open']=stock_data['Open'].transform(lambda x: x.ffill())
        stock_data['High']=stock_data['High'].transform(lambda x: x.ffill())
        stock_data['Low']=stock_data['Low'].transform(lambda x: x.ffill())
        stock_data['Close']=stock_data['Close'].transform(lambda x: x.ffill())
        stock_data['Adj Close']=stock_data['Adj Close'].transform(lambda x: x.ffill())
        stock_data['Volume']=stock_data['Volume'].transform(lambda x: x.ffill())

    #   add isin and name    
        comp_df=india_data[india_data['symbol']==sym]
        print(comp_df)
        comp_isin=comp_df['isin'].iloc[0]
        comp_name=comp_df['company_name'].iloc[0]

        stock_data['isin']=comp_isin
        stock_data['company']=comp_name

    #     rearrange isin and comapany name
        stock_data_new=stock_data.pop('company')
        stock_data.insert(0,'company',stock_data_new)

        stock_data_new=stock_data.pop('isin')
        stock_data.insert(0,'isin',stock_data_new)
        print('arranging the new dataframe')
        
        
    #     now delete useless cols 
        stock_data=stock_data.drop(['Volume','Adj Close'],axis=1)


    #     find the T day percentage change in close price
        perc_change=perc_return(stock_data)


    #     get last price
        last_price=stock_data.tail(1)['Close'].iloc[0]

    #     now add company name, isin , Close_today, perc_change in other dataframe(call it winner_losser_df)
        print('appending the list to resultant df')
        
        
        winner_loser_row=[comp_name,comp_isin,last_price,perc_change]
        winner_loser_df.append(winner_loser_row)

        print(i)
        i+=1
    
    except:
        data_not_found.append(sym)
    

    
#     pass this list to get a finalized data frame outside the loop

winner_loser_df=create_win_los_df(winner_loser_df)
winner_loser_df


In [ ]:
winner_loser_df=winner_loser_df.sort_values('%_change')
winner_loser_df.to_csv('India_results')